# Notebook API Python Spark GraphFrames

**GraphFrames Quick-Start Guide:**
*   https://graphframes.github.io/graphframes/docs/_site/quick-start.html

**GraphFrames User Guide:**
*   https://graphframes.github.io/graphframes/docs/_site/user-guide.html

**GraphFrames Python API**
*   https://graphframes.github.io/graphframes/docs/_site/api/python/index.html



***Install pyspark and findspark:***

***Install GraphFrames :***

In [1]:
import igraph as ig
import os
# !find /usr/local -name "pyspark"
os.environ["SPARK_HOME"] = "/home/anyes/spark"
os.environ["JAVA_HOME"] = "/usr"

***Start the spark session:***

In [2]:
# Main imports
import findspark
from pyspark.sql import SparkSession
from pyspark import SparkConf

# for dataframe and udf
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *

import pandas as pd


# initialise environment variables for spark
findspark.init()

# Start spark session
# --------------------------
def start_spark():
  local = "local[*]"
  appName = "TP"

  gf = "graphframes:graphframes:0.8.3-spark3.5-s_2.12"

  configLocale = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "6G").\
  set("spark.driver.memory","6G").\
  set("spark.sql.catalogImplementation","in-memory").\
  set("spark.jars.packages", gf)

  spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")

  spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")

  # Adjust the query execution environment to the size of the cluster (4 cores)
  spark.conf.set("spark.sql.shuffle.partitions","4")
  print("session started, its id is ", sc.applicationId)
  return spark
spark = start_spark()

24/02/06 10:56:25 WARN Utils: Your hostname, anyes-Latitude-5480 resolves to a loopback address: 127.0.1.1; using 10.51.22.213 instead (on interface wlp2s0)
24/02/06 10:56:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/anyes/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/anyes/.ivy2/cache
The jars for the packages stored in: /home/anyes/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5c273579-89cb-4f9e-8940-570359c240d0;1.0
	confs: [default]
	found graphframes#graphframes;0.8.3-spark3.5-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 446ms :: artifacts dl 14ms
	:: modules in use:
	graphframes#graphframes;0.8.3-spark3.5-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------

session started, its id is  local-1707213392001


In [3]:
#Import GraphFrames
from graphframes import GraphFrame
from graphframes.lib import AggregateMessages as AM
from graphframes.lib import Pregel

#For connectedComponents()
# !pwd
# !mkdir /content/checkpoints
spark.sparkContext.setCheckpointDir('./content/checkpoints')

#Import networkx
import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx

# Lecture des fichiers du dossier BDLE_1M

In [4]:
# Initialisez une session Spark
spark = SparkSession.builder.appName("Proteines_graph").getOrCreate()

parquet_folder = "./local/data/BDLE_10K"

# Récuperer tous les fichiers Parquet compressé avec Snappy
parquet_files = parquet_folder + "/*.snappy.parquet"

#Création du data frame
df = spark.read.format("parquet").option("compression", "snappy").load(parquet_files)

# Affichez les 10 premiére ligne du DataFrame
df.show(10)


+-----------+-----------+-----+
|     seqID1|     seqID2|  sim|
+-----------+-----------+-----+
|117761605:5|152890023:5| 97.8|
|152904885:3|155591878:2| 88.5|
|152887848:4|153682181:0|100.0|
|152937692:5| 80009514:2| 82.3|
|152990923:2|154549183:4| 98.0|
|152867782:1|153171917:1| 83.1|
| 15111981:2|153137370:1|100.0|
|152794195:0| 15280704:2| 96.9|
| 62963742:1| 63783418:5| 98.4|
|152170568:3|153062631:2| 97.6|
+-----------+-----------+-----+
only showing top 10 rows



In [5]:
# Nombre de lignes
nb_rows = df.count()

# Liste des noms des colonnes
columns_list = df.columns

# Nombre de colonnes
nb_columns = len(columns_list)

# Affichage des informations
print(f"Nombre de lignes : {nb_rows}")
print(f"Nombre de colonnes : {nb_columns}")
print(f"Noms des colonnes : {columns_list}")

Nombre de lignes : 20842
Nombre de colonnes : 3
Noms des colonnes : ['seqID1', 'seqID2', 'sim']


# Création d'un graphe avec Graphframes

In [6]:
# Créez un DataFrame distinct contenant tous les identifiants uniques de sommets
vertices = df.selectExpr("seqID1 as id").union(df.selectExpr("seqID2 as id")).distinct()

# Créez un DataFrame d'arêtes avec les colonnes "src", "dst", et "sim"
edges = df.select("seqID1", "seqID2", "sim").withColumnRenamed("seqID1", "src").withColumnRenamed("seqID2", "dst")


In [9]:
# Créez le GraphFrame
graph = GraphFrame(vertices, edges)
nombre_noeuds = graph.vertices.count()
nombre_aretes = graph.edges.count()

print("Nombre de noeuds :", nombre_noeuds)
print("Nombre d'arêtes :", nombre_aretes)

Nombre de noeuds : 18500
Nombre d'arêtes : 20842


# Prendre que les arêtes qui ont un pourcentage de similarité supérieure ou égale à 100

In [12]:
graph_100_edges = graph.edges.filter("sim >= 100")
graph_100 = GraphFrame(vertices, graph_100_edges)
graph_100 = graph_100.dropIsolatedVertices()

nombre_noeuds = graph_100.vertices.count()
nombre_aretes = graph_100.edges.count()

print("Nombre de noeuds :", nombre_noeuds)
print("Nombre d'arêtes :", nombre_aretes)

Nombre de noeuds : 5908
Nombre d'arêtes : 4054


# Calcul des composantes connexes

In [13]:
import time



start_time = time.time()
connected_components = graph_100.connectedComponents()
end_time = time.time()

print("Temps d'execution : ", end_time - start_time)

Temps d'execution :  162.0920000076294


**Strongly connected components**

In [ ]:
g.stronglyConnectedComponents(maxIter=10).orderBy("component").show()

+---+-------+---+---------+
| id|   name|age|component|
+---+-------+---+---------+
|  a|  Alice| 34|        0|
|  d|  David| 29|        0|
|  e| Esther| 32|        0|
|  b|    Bob| 36|        1|
|  c|Charlie| 30|        1|
|  f|  Fanny| 36|        5|
+---+-------+---+---------+

